As a Data Scientist, I was contacted by a friend of mine who wanted some advice on where to open a Mexican restaurant in Washington D.C. <br>
In order to help him I will be using Foursquare venues data, aqcuired by their API, to determine the best place to his new endeavor.  <br>
The intent is to see restaurants spread across 2 km of the city center and get to know what kind of dishes they have.

In [1]:
from bs4 import BeautifulSoup
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner

from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

import folium # plotting library

In [2]:
CLIENT_ID = 'C5FR1B1B0IHZX5B5WRHQZYDJHR1C4AEXC3CDKMLSXCJRMCG3'
CLIENT_SECRET = 'CJTR0CM40WD4UNBRUFSMQ3Q0TNTO5GVTM2E21KCVMGLPUVZA' 
ACCESS_TOKEN = '1LZAXTAX3TH0YIQFMAWHCQ4PKESV0AO1S3WMUDUPSU3P3ETR'
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

Retrieving the city coordinates

In [3]:
address = 'Washington DC, CO'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

38.8949924 -77.0365581


Using Foursquare API to filter restaurants around 5 km of the city center

In [4]:
search_query = 'Restaurant'
radius = 5000

In [5]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude,ACCESS_TOKEN, VERSION, search_query, radius, LIMIT)

In [6]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '6075b2787a96fb446aec559a'},
 'notifications': [{'type': 'notificationTray', 'item': {'unreadCount': 0}}],
 'response': {'venues': [{'id': '4bf1ee8da09076b0d7f329d4',
    'name': '1331 Restaurant & Lounge',
    'location': {'address': '1331 Pennsylvania Avenue NW',
     'crossStreet': 'btwn 13th & 14th St NW',
     'lat': 38.8963301,
     'lng': -77.0310183,
     'labeledLatLngs': [{'label': 'display',
       'lat': 38.8963301,
       'lng': -77.0310183}],
     'distance': 502,
     'postalCode': '20004',
     'cc': 'US',
     'city': 'Washington',
     'state': 'D.C.',
     'country': 'United States',
     'formattedAddress': ['1331 Pennsylvania Avenue NW (btwn 13th & 14th St NW)',
      'Washington, D.C. 20004',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d14e941735',
      'name': 'American Restaurant',
      'pluralName': 'American Restaurants',
      'shortName': 'American',
      'icon': {'prefix': 'https://ss3.4sqi.net/im

In [7]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

<ipython-input-7-5acf500bf9ad>:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  dataframe = json_normalize(venues)


,id,name,categories,referralId,hasPerk,location.address,location.crossStreet,location.lat,location.lng,location.labeledLatLngs,...,location.country,location.formattedAddress,delivery.id,delivery.url,delivery.provider.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.icon.name,venuePage.id,location.neighborhood
0,4bf1ee8da09076b0d7f329d4,1331 Restaurant & Lounge,"[{'id': '4bf58dd8d48988d14e941735', 'name': 'A...",v-1618326136,False,1331 Pennsylvania Avenue NW,btwn 13th & 14th St NW,38.896330,-77.031018,"[{'label': 'display', 'lat': 38.8963301, 'lng'...",...,United States,[1331 Pennsylvania Avenue NW (btwn 13th & 14th...,1493234,https://www.grubhub.com/restaurant/1331-bar--l...,grubhub,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_grubhub_20180129.png,NaN,NaN
1,45fec67bf964a5208c441fe3,Harry's Restaurant & Saloon,"[{'id': '4bf58dd8d48988d116941735', 'name': 'B...",v-1618326136,False,436 11th St NW,at E St. NW,38.895911,-77.027496,"[{'label': 'display', 'lat': 38.89591071893386...",...,United States,"[436 11th St NW (at E St. NW), Washington, D.C...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4bd181be77b29c74cda88c82,Harriet's Family Restaurant,"[{'id': '4bf58dd8d48988d143941735', 'name': 'B...",v-1618326136,False,436 11th St NW,at E St. NW,38.895910,-77.027042,"[{'label': 'display', 'lat': 38.89590976442353...",...,United States,"[436 11th St NW (at E St. NW), Washington, D.C...",1756674,https://www.grubhub.com/restaurant/harriets-fa...,grubhub,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",/delivery_provider_grubhub_20180129.png,NaN,NaN
3,4bd6e50c1728986e8d29b735,Morrison-Clark Historic Inn & Restaurant,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",v-1618326136,False,1011 L St NW,NaN,38.904234,-77.026694,"[{'label': 'display', 'lat': 38.90423359833188...",...,United States,"[1011 L St NW, Washington, D.C. 20001, United ...",NaN,NaN,NaN,NaN,NaN,NaN,154552768,NaN
4,4aa6f33bf964a5209f4b20e3,The Palm Restaurant,"[{'id': '4bf58dd8d48988d1cc941735', 'name': 'S...",v-1618326136,False,1225 19th St NW,at Jefferson Pl NW,38.906529,-77.043386,"[{'label': 'display', 'lat': 38.90652919805219...",...,United States,"[1225 19th St NW (at Jefferson Pl NW), Washing...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Removing some unnecessary columns.

In [8]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,id
0,1331 Restaurant & Lounge,American Restaurant,1331 Pennsylvania Avenue NW,btwn 13th & 14th St NW,38.896330,-77.031018,"[{'label': 'display', 'lat': 38.8963301, 'lng'...",502,20004,US,Washington,D.C.,United States,[1331 Pennsylvania Avenue NW (btwn 13th & 14th...,NaN,4bf1ee8da09076b0d7f329d4
1,Harry's Restaurant & Saloon,Bar,436 11th St NW,at E St. NW,38.895911,-77.027496,"[{'label': 'display', 'lat': 38.89591071893386...",791,20004,US,Washington,D.C.,United States,"[436 11th St NW (at E St. NW), Washington, D.C...",NaN,45fec67bf964a5208c441fe3
2,Harriet's Family Restaurant,Breakfast Spot,436 11th St NW,at E St. NW,38.895910,-77.027042,"[{'label': 'display', 'lat': 38.89590976442353...",830,20004,US,Washington,D.C.,United States,"[436 11th St NW (at E St. NW), Washington, D.C...",NaN,4bd181be77b29c74cda88c82
3,Morrison-Clark Historic Inn & Restaurant,Hotel,1011 L St NW,NaN,38.904234,-77.026694,"[{'label': 'display', 'lat': 38.90423359833188...",1337,20001,US,Washington,D.C.,United States,"[1011 L St NW, Washington, D.C. 20001, United ...",NaN,4bd6e50c1728986e8d29b735
4,The Palm Restaurant,Steakhouse,1225 19th St NW,at Jefferson Pl NW,38.906529,-77.043386,"[{'label': 'display', 'lat': 38.90652919805219...",1413,20036,US,Washington,D.C.,United States,"[1225 19th St NW (at Jefferson Pl NW), Washing...",NaN,4aa6f33bf964a5209f4b20e3
5,Stan's Restaurant & Lounge,American Restaurant,1029 Vermont Ave NW,NaN,38.903516,-77.032997,"[{'label': 'display', 'lat': 38.903516, 'lng':...",997,20005,US,Washington,D.C.,United States,"[1029 Vermont Ave NW, Washington, D.C. 20005, ...",NaN,40b13b00f964a52045f31ee3
6,Tony Cheng's Restaurant,Dim Sum Restaurant,619 H St NW,NaN,38.899827,-77.020841,"[{'label': 'display', 'lat': 38.89982732802529...",1464,20001,US,Washington,D.C.,United States,"[619 H St NW, Washington, D.C. 20001, United S...",NaN,40d38200f964a52048011fe3
7,BAO Restaurant & Lounge,Chinese Restaurant,1018 Vermont Ave NW,NaN,38.903255,-77.033806,"[{'label': 'display', 'lat': 38.90325546264648...",950,20005,US,Washington,D.C.,United States,"[1018 Vermont Ave NW, Washington, D.C. 20005, ...",NaN,4b213d32f964a520c83824e3
8,Chalin's Restaurant,Chinese Restaurant,1912 I St NW,btwn 19th & 20th Sts NW,38.901336,-77.044103,"[{'label': 'display', 'lat': 38.90133602105807...",962,20006,US,Washington,D.C.,United States,"[1912 I St NW (btwn 19th & 20th Sts NW), Washi...",NaN,4bc4d92f4cdfc9b6a3409921
9,Carmine’s Italian Restaurant,Italian Restaurant,425 7th St NW,NaN,38.895685,-77.021766,"[{'label': 'display', 'lat': 38.8956851, 'lng'...",1283,20004,US,Washington,D.C.,United States,"[425 7th St NW, Washington, D.C. 20004, United...",NaN,4c3b50185810a593cb73ba3c


Creating a map the restaurants found in Foursquare data

In [9]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13)

# add the restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

In [20]:
dataframe_filtered['categories'].value_counts()

American Restaurant          13
Latin American Restaurant     5
Chinese Restaurant            3
Irish Pub                     2
Vietnamese Restaurant         2
Thai Restaurant               2
New American Restaurant       2
Food                          2
Restaurant                    2
Office                        2
Ethiopian Restaurant          1
Italian Restaurant            1
Breakfast Spot                1
Strip Club                    1
Dive Bar                      1
Mexican Restaurant            1
Bar                           1
Café                          1
Seafood Restaurant            1
Steakhouse                    1
Hotel                         1
Dim Sum Restaurant            1
Sushi Restaurant              1
Name: categories, dtype: int64

We can see there are 5 Latin American and 1 Mexican restaurant that could be potential competitors

In [32]:
dataframe_filtered2 = dataframe_filtered[dataframe_filtered['categories'].str.contains('Latin|Mexican', na=False, regex=True)].reset_index(drop=True)
dataframe_filtered2

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,id
0,Izalco Bar Y Restaurant,Latin American Restaurant,1228 11th St NW,NaN,38.906544,-77.027390,"[{'label': 'entrance', 'lat': 38.906576, 'lng'...",1511,20001,US,Washington,D.C.,United States,"[1228 11th St NW, Washington, D.C. 20001, Unit...",NaN,4bfd8243f61dc9b6c7219fde
1,Restaurant Judy,Latin American Restaurant,2212 14th St NW,btwn Florida Ave & W St NW,38.919609,-77.032177,"[{'label': 'display', 'lat': 38.91960942517024...",2766,20009,US,Washington,D.C.,United States,"[2212 14th St NW (btwn Florida Ave & W St NW),...",U-Street,4b550d30f964a52054d927e3
2,Alero Restaurant,Mexican Restaurant,1724 Connecticut Ave NW,NaN,38.913532,-77.045722,"[{'label': 'display', 'lat': 38.9135322857215,...",2211,20009,US,Washington,D.C.,United States,"[1724 Connecticut Ave NW, Washington, D.C. 200...",NaN,4dab960c93a04642f0999175
3,Marleny's Restaurant,Latin American Restaurant,3201 Mount Pleasant St NW,at Lamont St NW,38.931292,-77.038118,"[{'label': 'display', 'lat': 38.931292, 'lng':...",4043,20010,US,Washington,D.C.,United States,"[3201 Mount Pleasant St NW (at Lamont St NW), ...",NaN,4b986eb7f964a520fb4135e3
4,Ercilia's Restaurant,Latin American Restaurant,3070 Mount Pleasant St NW,at Irving St NW,38.928654,-77.037439,"[{'label': 'display', 'lat': 38.92865427834165...",3747,20009,US,Washington,D.C.,United States,"[3070 Mount Pleasant St NW (at Irving St NW), ...",NaN,4ac78947f964a520d3b720e3
5,Don Juan Restaurant,Latin American Restaurant,1660 Lamont St NW,at Mt Pleasant St NW,38.931203,-77.038389,"[{'label': 'display', 'lat': 38.93120325176272...",4034,20010,US,Washington,D.C.,United States,"[1660 Lamont St NW (at Mt Pleasant St NW), Was...",Mount Pleasant,4b4e8a66f964a520e9f026e3


Creating a map with only Latin American and Mexican restaurants

In [34]:
venues_map2 = folium.Map(location=[latitude, longitude], zoom_start=13)

# add the restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered2.lat, dataframe_filtered2.lng, dataframe_filtered2.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map2)

# display map
venues_map2

We can see there are barely no direct competitor closer to the city center, so a good way to determine the location of our restaurant could be achieved by clustering (k-means algorithm) all venues from 1 km of the city center and rank the top 10 venues in this clusters.